In [3]:
import requests
import defs
import pandas as pd
import pickle
#from tabulate import tabulate

In [4]:
session = requests.Session()

In [5]:
Index = ['SPX500_USD'] #The API name for retreiving S & P 500 index data from the Oanda website.

In [6]:
def fetch_candles(Index_name, count, granularity): #Function returns the data from the Oanda website based on Index type
    url = f"{defs.OANDA_URL}/instruments/{Index_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params = params, headers = defs.SECURE_HEADER)
    return response.status_code, response.json()

In [7]:
def get_price(dictionary,prices,ohlc,candle): #Function returns the dictionary for candles for mid, ask and bid prices.
    for price in prices:
        for oh in ohlc:
            dictionary[f"{price}_{oh}"] = candle[price][oh]

In [8]:
def get_candles_df(json_response):# Creates a dataframe for the candles data on a given Index type
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o','h','l','c']
    
    our_data= []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        price = get_price(new_dict,prices,ohlc,candle)
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [9]:
def save_file(candles_df, Index_type, granularity):#Saves Files to a pickle document
    candles_df.to_pickle(f"{Index_type}_{granularity}.pkl")

In [10]:
def create_data(Index_type, count, granularity):
    code, json_data = fetch_candles(Index_type,count,granularity)
    if code != 200:
        print(pair,"Error")
        return
    df = get_candles_df(json_data)
    print(df)
    print(f"{Index_type} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df,Index_type,granularity)

In [13]:
try:
    create_data(Index[0],10,"D")
except:
    print("Caught it!")

                             time  volume   mid_o   mid_h   mid_l   mid_c  \
0  2021-05-02T21:00:00.000000000Z   59419  4192.4  4211.4  4190.2  4192.6   
1  2021-05-03T21:00:00.000000000Z  112144  4192.8  4194.3  4129.6  4167.0   
2  2021-05-04T21:00:00.000000000Z   60654  4167.8  4188.8  4162.0  4167.0   
3  2021-05-05T21:00:00.000000000Z  103930  4168.1  4205.3  4148.6  4204.6   
4  2021-05-06T21:00:00.000000000Z   73910  4205.2  4239.8  4199.4  4234.0   
5  2021-05-09T21:00:00.000000000Z   85520  4234.5  4245.4  4179.9  4183.4   
6  2021-05-10T21:00:00.000000000Z  192491  4186.4  4192.8  4111.4  4147.6   
7  2021-05-11T21:00:00.000000000Z  223432  4147.5  4157.3  4058.0  4058.7   
8  2021-05-12T21:00:00.000000000Z  213659  4061.2  4132.8  4035.6  4117.0   
9  2021-05-13T21:00:00.000000000Z   76524  4116.0  4184.0  4111.2  4179.0   

    bid_o   bid_h   bid_l   bid_c   ask_o   ask_h   ask_l   ask_c  
0  4192.1  4211.2  4190.0  4192.4  4192.7  4211.5  4190.3  4192.8  
1  4192.6  4194.

In [12]:
with open("SPX500_USD_H1.pkl", "rb") as f:
    object = pickle.load(f)
    
df = pd.DataFrame(object)
df.to_csv(r'SPX500_Data.csv')

ValueError: unsupported pickle protocol: 5